In [1]:
from twikit import Client
import time
import json
from twikit import TwitterException 
import pandas as pd

# this API requires authentication
f = open('authentication.txt', 'r')
auth = f.read()
f.close()
auth_token = auth.split("\n")

# don't hardcode your email and password into something!!!
# the auth is in gitignore so I won't get hacked
USERNAME = str(auth_token[0])
EMAIL = str(auth_token[1])
PASSWORD = str(auth_token[2])

# Initialize client
client = Client(language='en-US', http2=True)

# Login to the service with provided user credentials
client.login(
    auth_info_1=USERNAME ,
    auth_info_2=EMAIL,
    password=PASSWORD
)

C:\Users\eaesp\AppData\Local\Temp\ipykernel_31180\2204781778.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


{'flow_token': 'g;171368131064059543:-1713681310699:8lWqsvisG6GkvlXFNI5AbNLc:14',
 'status': 'success',
 'subtasks': [{'subtask_id': 'LoginSuccessSubtask',
   'open_account': {'user': {'id': 1547081484695216130,
     'id_str': '1547081484695216130',
     'name': 'Eloragh Espie',
     'screen_name': 'EloraghEspie'},
    'next_link': {'link_type': 'subtask',
     'link_id': 'next_link',
     'subtask_id': 'SuccessExit'},
    'attribution_event': 'login'}},
  {'subtask_id': 'SuccessExit',
   'open_link': {'link': {'link_type': 'subtask',
     'link_id': 'next_link',
     'subtask_id': 'LoginOpenHomeTimeline'}}},
  {'subtask_id': 'LoginOpenHomeTimeline',
   'open_home_timeline': {'next_link': {'link_type': 'abort',
     'link_id': 'next_link'}}}]}

In [2]:
# Twitter LOVES to ban people when they log in repeatedly
# saving the cookies makes sure I don't get banned (often)

client.get_cookies()
client.save_cookies('cookies.json')
with open('cookies.json', 'r', encoding='UTF8') as f:
    client.set_cookies(json.load(f))

In [30]:
from twikit import UserUnavailable
from twikit import UserNotFound

def get_user_id(num, handle):
    # load the cookies so you don't login a million times and get banned
    client.load_cookies('cookies.json')
    
    # pull IDs and insert them back into the list
    try:
        user_id = client.get_user_by_screen_name(handle).id
        print(f'{num} Success {handle}, {user_id}')

    # if user is inaccessible these will keep the loop from breaking
    except UserUnavailable:
        print(f"{num} User {handle} unavailable")
    except UserNotFound:
        print(f'{num} User {handle} not found')

    return(user_id)

In [6]:
def create_key(user_id, year):
    # load the cookies so you don't login a million times and get banned
    client.load_cookies('cookies.json')
    return(user_id + year)

In [34]:
import time
from math import ceil
import requests
from twikit import TooManyRequests
from twikit.utils import Endpoint

# this is a housekeeping function
# twitter API can throw rate limits
# they're kind of like timeouts
# this function just shows me how much longer I will be in timeout for

def get_limit_reset_time(endpoint: str):
    res = requests.get(
        endpoint,
        headers=client._base_headers,
        cookies=client.get_cookies()
    )
    return ceil(int(res.headers['x-rate-limit-reset']) - time.time())

In [36]:
# timeout check for accessing user IDs
try:
    print(client.get_user_by_screen_name('BarackObama'))
except TooManyRequests:
    
    reset_time = get_limit_reset_time(Endpoint.USER_TWEETS)
    print(f'rate limit is reset after {reset_time} seconds.')

rate limit is reset after 898 seconds.


In [19]:
data = pd.read_csv("data/twitter politicians.csv", keep_default_na=False)
df = pd.DataFrame(data)

In [20]:
# turn pandas into list
# there is probably a better way to do this but this works 
data_list = []
for index, row in df.iterrows():

    row_list = df.loc[index, :].values.flatten().tolist()
    data_list.append(row_list)
    
print(data_list)

[['', 'Barack Obama', 'BarackObama'], ['', 'Joe Biden', 'JoeBiden'], ['', 'Dennis Kucinich', 'Dennis_Kucinich'], ['', 'Bill Richardson', 'GovRichardson'], ['', 'Mike Huckabee', 'GovMikeHuckabee'], ['', 'Hillary Clinton', 'HillaryClinton'], ['', 'Ron Paul', 'RonPaul'], ['', 'John McCain', 'SenJohnMcCain'], ['', 'Newt Gingrich', 'newtgingrich'], ['', 'Mike Gravel', 'MikeGravel_US'], ['', 'Alan Keyes', 'loyaltoliberty'], ['', 'Mitt Romney', 'MittRomney'], ['', 'Fred Thompson', 'fredthompson'], ['', 'Chris Dodd', 'SenChrisDodd'], ['', 'Rudy Guiliani', 'RudyGiuliani'], ['', 'Ralph Nader', 'RalphNader'], ['', 'Tom Tancredo', 'ttancredo'], ['', 'Jill Stein', 'DrJillStein'], ['', 'Virgil Goode', 'VirgilGoode'], ['', 'Gary Johnson', 'GovGaryJohnson'], ['', 'Rick Santorum', 'RickSantorum'], ['', 'Hllary Clinton', 'HillaryClinton'], ['', 'Bernie Sanders', 'SenSanders'], ['', 'Donald Trump', 'realDonaldTrump'], ['', 'Jeb Bush', 'JebBush'], ['', 'Ted Cruz', 'TedCruz'], ['', 'Marco Rubio', 'MarcoRub

In [28]:
batch1 = [item for item in data_list[0:50]]
batch2 = [item for item in data_list[50:100]]
batch3 = [item for item in data_list[100:150]]
batch4 = [item for item in data_list[150:]]

mass_list = [batch1, batch2, batch3, batch4]

In [22]:
print(batch1)

[['', 'Barack Obama', 'BarackObama'], ['', 'Joe Biden', 'JoeBiden'], ['', 'Dennis Kucinich', 'Dennis_Kucinich'], ['', 'Bill Richardson', 'GovRichardson'], ['', 'Mike Huckabee', 'GovMikeHuckabee'], ['', 'Hillary Clinton', 'HillaryClinton'], ['', 'Ron Paul', 'RonPaul'], ['', 'John McCain', 'SenJohnMcCain'], ['', 'Newt Gingrich', 'newtgingrich'], ['', 'Mike Gravel', 'MikeGravel_US'], ['', 'Alan Keyes', 'loyaltoliberty'], ['', 'Mitt Romney', 'MittRomney'], ['', 'Fred Thompson', 'fredthompson'], ['', 'Chris Dodd', 'SenChrisDodd'], ['', 'Rudy Guiliani', 'RudyGiuliani'], ['', 'Ralph Nader', 'RalphNader'], ['', 'Tom Tancredo', 'ttancredo'], ['', 'Jill Stein', 'DrJillStein'], ['', 'Virgil Goode', 'VirgilGoode'], ['', 'Gary Johnson', 'GovGaryJohnson'], ['', 'Rick Santorum', 'RickSantorum'], ['', 'Hllary Clinton', 'HillaryClinton'], ['', 'Bernie Sanders', 'SenSanders'], ['', 'Donald Trump', 'realDonaldTrump'], ['', 'Jeb Bush', 'JebBush'], ['', 'Ted Cruz', 'TedCruz'], ['', 'Marco Rubio', 'MarcoRub

In [32]:
i = 1
final_list = []

for lst in mass_list:
    print(i)
    j = 0
    for item in lst:
        user_id = get_user_id(j, item[2])
        item[0] = user_id
        final_list.append(item)
        time.sleep(5) # cooldown to avoid rate limits
        j += 1
    i += 1
    print("")
    time.sleep(120) # cooldown to avoid rate limits

1
0 Success BarackObama, 813286
1 Success JoeBiden, 939091
2 Success Dennis_Kucinich, 15022633
3 Success GovRichardson, 31428685
4 Success GovMikeHuckabee, 15416505
5 Success HillaryClinton, 1339835893
6 Success RonPaul, 287413569
7 Success SenJohnMcCain, 19394188
8 Success newtgingrich, 20713061
9 Success MikeGravel_US, 752638969095999489
10 Success loyaltoliberty, 19637821
11 Success MittRomney, 50055701
12 Success fredthompson, 2704951
13 Success SenChrisDodd, 16317406
14 Success RudyGiuliani, 770781940341288960
15 Success RalphNader, 77314692
16 Success ttancredo, 64534908
17 Success DrJillStein, 111216929
18 Success VirgilGoode, 605641396
19 Success GovGaryJohnson, 95713333
20 Success RickSantorum, 58379000
21 Success HillaryClinton, 1339835893
22 Success SenSanders, 29442313
23 Success realDonaldTrump, 25073877
24 Success JebBush, 113047940
25 Success TedCruz, 23022687
26 Success MarcoRubio, 15745368
27 Success JohnKasich, 18020081
28 Success SenBobCorker, 73303753
29 Success Mik

TooManyRequests: status: 429, message: "Rate limit exceeded
"

In [ ]:
final_list_copy = final_list.copy()

In [ ]:
i = 1
for item in final_list:
    key = str(i).rjust(6, '0')
    print(key)
    item.insert(0, key)
    i += 1

In [ ]:
print(final_list)

In [34]:
import sqlite3
conn = sqlite3.connect('tweets.db')
c = conn.cursor()

c.execute("""CREATE TABLE politicians (
          key INTEGER primary key,
          twitter_user_id STRING,
          politician_name STRING,
          twitter_handle STRING
          )""")

In [36]:
import sqlite3
conn = sqlite3.connect('tweets.db')
c = conn.cursor()

c.executemany("INSERT INTO politicians VALUES (?,?,?)", final_list)
conn.commit()

In [18]:
print(final_list)

[['813286', 'BarackObama'], ['939091', 'JoeBiden'], ['15022633', 'Dennis_Kucinich'], ['31428685', 'GovRichardson'], ['15416505', 'GovMikeHuckabee'], ['1339835893', 'HillaryClinton'], ['287413569', 'RonPaul'], ['19394188', 'SenJohnMcCain'], ['20713061', 'newtgingrich'], ['752638969095999489', 'MikeGravel_US'], ['19637821', 'loyaltoliberty'], ['50055701', 'MittRomney'], ['2704951', 'fredthompson'], ['16317406', 'SenChrisDodd'], ['770781940341288960', 'RudyGiuliani'], ['77314692', 'RalphNader'], ['64534908', 'ttancredo'], ['111216929', 'DrJillStein'], ['605641396', 'VirgilGoode'], ['95713333', 'GovGaryJohnson'], ['58379000', 'RickSantorum'], ['1339835893', 'HillaryClinton'], ['29442313', 'SenSanders'], ['25073877', 'realDonaldTrump'], ['113047940', 'JebBush'], ['23022687', 'TedCruz'], ['15745368', 'MarcoRubio'], ['18020081', 'JohnKasich'], ['73303753', 'SenBobCorker'], ['22203756', 'Mike_Pence'], ['2987671552', 'GovLarryHogan'], ['342863309', 'BetoORourke'], ['22044727', 'SenAmyKlobuchar'

In [ ]:
i = 1
for item in final_list:
    key = str(i).rjust(6, '0')
    item.insert(0, key)
    print(item)
    i += 1

In [2]:
import sqlite3
conn = sqlite3.connect('tweets.db')
c = conn.cursor()

c.execute("""CREATE TABLE politicians (
          twitter_user_id STRING,
          politician_name STRING,
          twitter_handle STRING
          )""")

OperationalError: table politicians already exists

In [24]:
conn.execute('DROP TABLE politicians')

NameError: name 'conn' is not defined

In [ ]:
c.executemany("INSERT INTO tweets VALUES (?,?,?,?,?,?)", final_list)
conn.commit()

In [5]:
import sqlite3
conn = sqlite3.connect('tweets.db')
c = conn.cursor()

c.execute("""CREATE TABLE coordinates (
          key INTEGER primary key,
          twitter_user_id STRING,
          politician_name STRING,
          twitter_handle STRING,
          x_coordinate INTEGER,
          y_coordinate INTEGER,
          political_party STRING,
          election_year INTEGER,
          country STRING,
          twitter_active_during_election STRING
          )""")

In [4]:
conn.execute('DROP TABLE coordinates')

In [7]:
data = pd.read_csv("data/non-twitter politicians.csv", keep_default_na=False)
df = pd.DataFrame(data)

# turn pandas into list
# there is probably a better way to do this but this works 
data_list = []
for index, row in df.iterrows():

    row_list = df.loc[index, :].values.flatten().tolist()
    data_list.append(row_list)
    
print(data_list)

[[2008, 'null', 'Duncan Hunter', 'null', 7, 6, 'Republican', 2008, 'USA', False], [12008, 'null', 'John Edwards', 'null', 2, 1, 'Democratic', 2008, 'USA', False], [12012, 'null', 'Stewart Alexander', 'null', -5, -1, 'Socialist', 2012, 'USA', False], [22012, 'null', 'Rockey Anderson', 'null', 4, -2, 'Justice', 2012, 'USA', False], [22008, 'null', 'Stephane Dion', 'null', 2, 3, 'Liberal', 2008, 'CA', False], [32008, 'null', 'Gilles Duceppe', 'null', -1, -1, 'Bloc Quebecios', 2008, 'CA', False], [12013, 'null', 'Angela Merkel', 'null', 7, 6, 'CDU/SCU', 2013, 'DE', False], [22013, 'null', 'Rainer Bruderle', 'null', 8, -1, 'FDP', 2013, 'DE', False], [12010, 'null', 'Tony Crook', 'null', 5, 6, 'WA Nationals', 2010, 'AUS', False], [12011, 'null', 'Tariana Turia', 'null', -1, 0, 'Maori Party', 2011, 'NZ', False], [12017, 'null', 'Marama Fox', 'null', -1, 0, 'Maori Party', 2017, 'NZ', False]]


In [15]:
for item in data_list[1]:
    print(item, type(item))

12008 <class 'numpy.int64'>
null <class 'str'>
John Edwards <class 'str'>
null <class 'str'>
2 <class 'numpy.int64'>
1 <class 'numpy.int64'>
Democratic <class 'str'>
2008 <class 'numpy.int64'>
USA <class 'str'>
False <class 'numpy.bool_'>


In [8]:
import numpy as np
sqlite3.register_adapter(np.int64, int)
sqlite3.register_adapter(np.bool_, str)

In [9]:
c.executemany("INSERT INTO coordinates VALUES (?,?,?,?,?,?,?,?,?,?)", data_list)
conn.commit()